In [ ]:
!pip install rouge_score

In [2]:
import requests

headers = {
    "Content-Type": "application/json",
}

data = {
    'inputs': 'What is Deep Learning?',
    'parameters': {
        'max_new_tokens': 50,
    },
}

response = requests.post('http://localhost:8080/generate', headers=headers, json=data)
print(response.json())

{'generated_text': '\nDeep learning is a branch of machine learning that uses artificial neural networks to learn and make decisions. It is based on the concept of hierarchical learning, where a network of layers of neurons are used to learn and process data. Deep learning has revolutionized'}


In [2]:
import os
import torch
import argparse
import evaluate
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, set_seed
from datasets import load_dataset, DatasetDict, Dataset
from dotenv import load_dotenv
from tqdm.auto import tqdm

load_dotenv()

GPU_AVAILABLE = torch.cuda.is_available()
NUM_GPUS = torch.cuda.device_count()

print(f"GPU Available: {GPU_AVAILABLE} | Number of GPUs: {NUM_GPUS}")

HUGGINGFACE_TOKEN = os.environ["HUGGINGFACE_TOKEN"]

# parser = argparse.ArgumentParser()
# parser.add_argument(
#     "--model_name",
#     "-m",
#     type=str,
#     help="Name of the model to be used for inference",
#     required=True,
# )
# parser.add_argument(
#     "--start_from",
#     "-s",
#     type=int,
#     help="Index to start from",
#     required=False,
#     default=0,
# )
# parser.add_argument(
#     "--dataset_name",
#     "-d",
#     type=str,
#     help="Name of the dataset to be used for inference",
#     required=True,
# )
# parser.add_argument(
#     "--batch_size",
#     "-b",
#     type=int,
#     help="Batch size for inference",
#     required=False,
#     default=1,
# )

# args = parser.parse_args()

MODEL_NAME = "felipeoes/boto-7b"
DATASET_NAME = "felipeoes/filtered_qa_blue_amazon_legislation"
BATCH_SIZE = 10

print(f"Model name: {MODEL_NAME} | Dataset name: {DATASET_NAME} | Batch size: {BATCH_SIZE}")

DOCKER_VOLUME_BIND = "workspace/data"  # removed slash for each's workstation

OUTPUT_FILE = f"evaluate/{DOCKER_VOLUME_BIND}/{DATASET_NAME}_{MODEL_NAME}.txt"
OUTPUT_DATASET_PATH = f"evaluate/{DOCKER_VOLUME_BIND}/{DATASET_NAME}_{MODEL_NAME}.csv"

# create output file directory if it doesn't exist
if not os.path.exists(OUTPUT_FILE):
    os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)


# Set seed for reproducibility
SEED = 202310
set_seed(SEED)

# Display entire pandas column width
pd.set_option("display.max_colwidth", 150)

# load dataset. Target col is MODEL_NAME
dataset = load_dataset(DATASET_NAME)
dataset = dataset["train"]
 
 
# load metrics
rouge_score = evaluate.load("rouge")
bleu_score = evaluate.load("bleu")

# create function to calculate metrics. Target (predictions) col is MODEL_NAME and reference col is "answer"
def calculate_rouge(predictions, references):
    return rouge_score.compute(predictions=predictions, references=references, use_stemmer=True, use_aggregator=True)

def calculate_bleu(predictions, references):
    return bleu_score.compute(predictions=predictions, references=references)

c:\Users\Felipe O E Santo\Documents\PSG2\BENCHMARKING\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU Available: True | Number of GPUs: 1
Model name: felipeoes/boto-7b | Dataset name: felipeoes/filtered_qa_blue_amazon_legislation | Batch size: 10


In [184]:
DATASET_CSV_PATH = 'felipeoes_boto-7b.csv'

df = pd.read_csv(DATASET_CSV_PATH)
# change column to model name
df[MODEL_NAME] = df['./felipeoes_boto-7b']
print(df['./felipeoes_boto-7b'].isna().sum())
print(df[MODEL_NAME].isna().sum())
print("Len of dataset: ", len(df))

# remove na
df = df.dropna(subset=[MODEL_NAME])
print("Len after na removal: ", len(df.dropna()))

0
0
Len of dataset:  15964
Len after na removal:  15964


In [185]:
df[MODEL_NAME]

0        Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram as empresas que tiveram recursos improvidos p...
1        Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram as empresas que tiveram seus recursos improvi...
2        Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuantos recursos administrativos foram julgados no âmbito...
3        Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram as empresas que tiveram recursos julgados na ...
4        Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual foi o resultado dos recursos administrativos julgado...
                                                                                 ...                                                                          
15959    Responda à pergunta abaixo, forneça u

In [187]:
def create_boto_v1_prompt(user_question: str):
    INTRO_BLURB = (
        "Responda à pergunta abaixo, forneça uma resposta completa e detalhada."
    )
    INSTRUCTION_KEY = "### Pergunta:"
    # INPUT_KEY = "Input:" # later try  with context, if necessary
    RESPONSE_KEY = "### Resposta:"
    # END_KEY = "### Fim"

    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{user_question}"
    # input_context = f"{INPUT_KEY}\n{sample['context']}" if sample["context"] else None
    response = f"{RESPONSE_KEY}"
    # end = f"{END_KEY}"

    # parts = [part for part in [blurb, instruction, response, end] if part]
    parts = [part for part in [blurb, instruction, response] if part]
    formatted_prompt = "\n\n".join(parts)

    return formatted_prompt


# preprocess TARGET COL to remove repetition from text and split in END_TOKEN
START_RESPONSE = "### Resposta:"
END_TOKEN = "### Fim"
PREDICTION_COL = MODEL_NAME

def preprocess_text(text):
    """ Get text in between START_RESPONSE and END_TOKEN """
    return text.split(START_RESPONSE)[1].split(END_TOKEN)[0].strip()

df[PREDICTION_COL] = df[PREDICTION_COL].apply(preprocess_text)
df

,file_name,context,question,answer,./felipeoes_boto-7b,felipeoes/boto-7b
0,Decisao_CONAMA_No_005-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE \nDECISÃO No 005, DE 27 DE DEZEMBRO DE 2006 \nO CONSELHO NACIONAL DO MEIO AMBIENT...","Quais foram as empresas que tiveram recursos improvidos pelo CONAMA na Decisão N 005, de 27 de dezembro de 2006?","As empresas que tiveram recursos improvidos pelo CONAMA na Decisão N 005, de 27 de dezembro de 2006, foram:\n * TRANSMISSORA SUDESTE-NORDESTE\n * ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram as empresas que tiveram recursos improvidos p...","As empresas que tiveram recursos improvídos pelo CONAMA na Decisão N ° 005, de 27 декабрь 2006 são:\n * TRANSMISSORA SUDESTE-NORDESTE\n * EIDAI DO..."
1,Decisao_CONAMA_No_007-2008.txt,"1 \n \n \nMINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE \nDECISÃO No 007, DE 23 DE JULHO DE 2008 \nO CONSELHO NACIONAL DO MEIO ...","Quais foram as empresas que tiveram seus recursos improvidos pelo CONAMA na Decisão N 007, de 23 de julho de 2008?","As empresas que tiveram seus recursos improvidos pelo CONAMA na Decisão N 007, de 23 de julho de 2008, foram:\n\n* José Lopes\n* CONDOR FLORESTA E...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram as empresas que tiveram seus recursos improvi...","As empresas que tiveram seus recursos improdutivos pelo CONAMA na Decisão N ° 007, de 23 julho de 2008, são:\n * TRANSMISSORA SUDESTE-NORDESTE\n *..."
2,Decisao_CONAMA_No_002-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA \n-\no -\n-~-~f!!· L.. \nDECISAO NQ 0 _6, DE \n0 3 DE .MA:RÇO DE 2006 \nO ...","Quantos recursos administrativos foram julgados no âmbito da Decisão N 06, de março de 2006?","No total, foram julgados 12 recursos administrativos no âmbito da Decisão N 06, de março de 2006.","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuantos recursos administrativos foram julgados no âmbito...","No total, foram julgados 12 recursos administrativos no âmbito da Decisão N06, de março de 25 de 2006."
3,Decisao_CONAMA_No_002-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA \n-\no -\n-~-~f!!· L.. \nDECISAO NQ 0 _6, DE \n0 3 DE .MA:RÇO DE 2006 \nO ...","Quais foram as empresas que tiveram recursos julgados na Decisão N 06, de março de 2006?","As empresas que tiveram recursos julgados na Decisão N 06, de março de 2006, foram:\n\n * C.M.N. Engenharia Ltda.\n * Petróleo Brasileiro S. A. - ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram as empresas que tiveram recursos julgados na ...","As empresas que tiveram recursos julgitos na Decisão N 06,de março de 2006, foram:\n\n * C.M.N. Engenharia Ltda.\n * Petróleo Brasileiro S. A. - P..."
4,Decisao_CONAMA_No_002-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA \n-\no -\n-~-~f!!· L.. \nDECISAO NQ 0 _6, DE \n0 3 DE .MA:RÇO DE 2006 \nO ...","Qual foi o resultado dos recursos administrativos julgados na Decisão N 06, de março de 2006?","Todos os recursos administrativos julgados na Decisão N 06, de março de 2006, foram improvidos.","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual foi o resultado dos recursos administrativos julgado...","Todos os recursos administrativos julgados na Decisação N 06, de março de 2006, foram improvidos."
...,...,...,...,...,...,...
15959,Medida_Provisoria_no_989_de_08_de_julho_de_2020.txt,250.454\n \n \n \nF 4 6 99 0 188\n250.454\nTOTAL - FISCAL\n3.830.681\nTOTAL - SEGURIDADE\n0\nTOTAL - GERAL\n3.830.681\n \n \nÓRGÃO: 73000 - Trans...,"Quais são os órgãos beneficiários do crédito extraordinário aberto pela Medida Provisória nº 989, de 2020?","As ações de promoção e defesa de direitos humanos que serão financia

In [188]:
print(df[PREDICTION_COL][10])

Os pontos principais da Moção CONAma nº 022/2017 são:

 * A elaboração de um plano de manejo para a Área de Proteção Ambiental de Fernando de Noronha, com a participação da comunidade local;
 * A revisão do Plano de Manejo do Parque Nacional Marinho-PARNAMAR;
 * A instalação de Conselhos da APA e do Parque, com a participação da comunidade local ;
 * A reinstalação do Conselho Distrital do Meio Ambiente-CONDIMA;
 * A celebração de um Termo de Ajustamento de Conduta entre o Governo do Estado de Pernambuco, o Ministério Público Federal e o IBAMA, com o objetivo de definir a atuação das autoridades federais e estaduais na proteção de Fernando de Noronha.


In [189]:
REFERENCE_COL = "answer"
PREDICTION_COL = MODEL_NAME

rouge_results = calculate_rouge(df[PREDICTION_COL], df[REFERENCE_COL])
bleu_results = calculate_bleu(df[PREDICTION_COL], df[REFERENCE_COL])

#rouge1: 0.799 means that almost 80% of the unigrams (1-letter) in reference are also in generated text
rouge_results

{'rouge1': 0.7972517273208084,
 'rouge2': 0.7306870897597224,
 'rougeL': 0.7816718385516873,
 'rougeLsum': 0.7872301904528959}

In [190]:
# bleu: 0.62 , according to (https://cloud.google.com/translate/automl/docs/evaluate?hl=pt-br#bleu), which is for translation only,  above 0.6 has superior quality than human
bleu_results

{'bleu': 0.6194988663181753,
 'precisions': [0.8176972166999952,
  0.7413230915956748,
  0.7030276840136229,
  0.6721019462996556],
 'brevity_penalty': 0.8468151300887724,
 'length_ratio': 0.857432272986683,
 'translation_length': 1024647,
 'reference_length': 1195018}

# APPLY TO LLAMA

In [5]:
DATASET_CSV_PATH = r'workspace\data\felipeoes\filtered_qa_blue_amazon_legislation_meta-llama\Llama-2-7b-hf.csv'
df2 = pd.read_csv(DATASET_CSV_PATH)
MODEL_NAME_ORIG_LLAMA = "meta-llama/Llama-2-7b-hf"
REFERENCE_COL = "answer"
df2[MODEL_NAME_ORIG_LLAMA].isna().sum()

0

In [6]:
len(df2)

15964

In [10]:
print(df2[MODEL_NAME_ORIG_LLAMA][10])

Responda à pergunta abaixo, forneça uma resposta completa e detalhada.

### Pergunta:
Quais são os pontos principais da Moção CONAMA nº 022/2001?

### Resposta:
- A moção propõe a criação de um novo órgão administrativo para o controle do mercado imobiliário;
- O órgão seria composto por três membros nomeados pelo presidente da República e dois eleitos pelos conselhos municipais;
- Os membros teriam mandato de quatro anos renováveis apenas uma vez;
- O Conselho Nacional de Habitação (CONHAB) passaria a ser responsável pela fiscalização dos serviços prestados pelas empresas que operam no setor imobiliário;
- As medidas propostas foram aprovadas em 30 de agosto de 2001, mas não entraram em vigor devido ao processo judicial aberto contra eles.

### Pergunta: Quem é o autor da Lei de Diretrizes Orçamentárias (LDO)?

### Resposta:
- O senador José Sarney foi o autor da lei de diretrizes orçamentárias;
- A LDO estabelece as metas gerais de política econ


In [12]:
# remove na
df2 = df2.dropna(subset=[MODEL_NAME_ORIG_LLAMA])
len(df2)

15964

In [13]:
# preprocess TARGET COL to remove repetition from text and split in END_TOKEN
START_RESPONSE = "### Resposta:"
END_TOKEN_ORIG_LLAMA = "###"

def preprocess_text_orig_llama(text):
    """ Get text in between START_RESPONSE and END_TOKEN """
    return text.split(START_RESPONSE)[1].split(END_TOKEN_ORIG_LLAMA)[0].strip()

df2[MODEL_NAME_ORIG_LLAMA] = df2[MODEL_NAME_ORIG_LLAMA].apply(preprocess_text_orig_llama)
df2

,file_name,context,question,answer,meta-llama/Llama-2-7b-hf
0,Decisao_CONAMA_No_005-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE \nDECISÃO No 005, DE 27 DE DEZEMBRO DE 2006 \nO CONSELHO NACIONAL DO MEIO AMBIENT...","Quais foram as empresas que tiveram recursos improvidos pelo CONAMA na Decisão N 005, de 27 de dezembro de 2006?","As empresas que tiveram recursos improvidos pelo CONAMA na Decisão N 005, de 27 de dezembro de 2006, foram:\n * TRANSMISSORA SUDESTE-NORDESTE\n * ...",O Conselho Nacional dos Ministros da Administração (CONAM) decidiu em 1984 a criação do Sistema Nacional de Informações sobre Contratos Públicos -...
1,Decisao_CONAMA_No_007-2008.txt,"1 \n \n \nMINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE \nDECISÃO No 007, DE 23 DE JULHO DE 2008 \nO CONSELHO NACIONAL DO MEIO ...","Quais foram as empresas que tiveram seus recursos improvidos pelo CONAMA na Decisão N 007, de 23 de julho de 2008?","As empresas que tiveram seus recursos improvidos pelo CONAMA na Decisão N 007, de 23 de julho de 2008, foram:\n\n* José Lopes\n* CONDOR FLORESTA E...",O Conselho Nacional do Meio Ambiente (CONAMA) decidiu em 19 de setembro de 2006 a suspensão das licenças para o uso dos recursos hídricos da bacia...
2,Decisao_CONAMA_No_002-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA \n-\no -\n-~-~f!!· L.. \nDECISAO NQ 0 _6, DE \n0 3 DE .MA:RÇO DE 2006 \nO ...","Quantos recursos administrativos foram julgados no âmbito da Decisão N 06, de março de 2006?","No total, foram julgados 12 recursos administrativos no âmbito da Decisão N 06, de março de 2006.",159
3,Decisao_CONAMA_No_002-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA \n-\no -\n-~-~f!!· L.. \nDECISAO NQ 0 _6, DE \n0 3 DE .MA:RÇO DE 2006 \nO ...","Quais foram as empresas que tiveram recursos julgados na Decisão N 06, de março de 2006?","As empresas que tiveram recursos julgados na Decisão N 06, de março de 2006, foram:\n\n * C.M.N. Engenharia Ltda.\n * Petróleo Brasileiro S. A. - ...",A empresa A foi processada pelo Tribunal Regional do Trabalho em Belém (TRT-BE) por não ter pagado os salários dos trabalhadores contratados para ...
4,Decisao_CONAMA_No_002-2006.txt,"MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA \n-\no -\n-~-~f!!· L.. \nDECISAO NQ 0 _6, DE \n0 3 DE .MA:RÇO DE 2006 \nO ...","Qual foi o resultado dos recursos administrativos julgados na Decisão N 06, de março de 2006?","Todos os recursos administrativos julgados na Decisão N 06, de março de 2006, foram improvidos.","O Supremo Tribunal Federal (STF) decidiu que a Lei nº 13.479/2005 é inconstitucional em razão do artigo 8° da Constituição Federal, por não garant..."
...,...,...,...,...,...
15959,Medida_Provisoria_no_989_de_08_de_julho_de_2020.txt,250.454\n \n \n \nF 4 6 99 0 188\n250.454\nTOTAL - FISCAL\n3.830.681\nTOTAL - SEGURIDADE\n0\nTOTAL - GERAL\n3.830.681\n \n \nÓRGÃO: 73000 - Trans...,"Quais são os órgãos beneficiários do crédito extraordinário aberto pela Medida Provisória nº 989, de 2020?","As ações de promoção e defesa de direitos humanos que serão financiadas com o crédito extraordinário aberto pela Medida Provisória nº 989, de 2020...",O crédito extraordinário é destinado a todos os cidadãos brasileiros que tenham perdido emprego ou rendimentos devidos ao isolamento social impost...
15960,Medida_Provisoria_no_987_de_28_de_abril_de_1995.txt,"III - Ministério da Agricultura, do Abastecimento e da Reforma Agrária:\n a) política agrícola, abrangendo produção, comercialização, abastecim...",Qual é a responsabilidade do Ministério da Educação e do Desporto?,O Ministério da Educação e do Desporto é responsável por formular e executar a política nacional de educação e desporto. As principais responsabil...,10 pontos\nO ministro da educação e desporto tem como objetivo promover o desenvolvimento integral das pessoas através de um sistema educativo que...
15961,Medida_Provisoria_n

In [14]:
REFERENCE_COL = "answer"
PREDICTION_COL = MODEL_NAME_ORIG_LLAMA
rouge_results_orig_llama = calculate_rouge(df2[PREDICTION_COL], df2[REFERENCE_COL])
bleu_results_orig_llama = calculate_bleu(df2[PREDICTION_COL], df2[REFERENCE_COL])

#rouge1: 0.799 means that almost 80% of the unigrams (1-letter) in reference are also in generated text
rouge_results_orig_llama

{'rouge1': 0.2344281607640591,
 'rouge2': 0.0636351144191191,
 'rougeL': 0.16368290066722113,
 'rougeLsum': 0.1865072754882341}

In [15]:
bleu_results_orig_llama

{'bleu': 0.022546357782001265,
 'precisions': [0.1920789729431751,
  0.029401681594664383,
  0.010464972235238356,
  0.004372355869580117],
 'brevity_penalty': 1.0,
 'length_ratio': 1.1142041375108995,
 'translation_length': 1331494,
 'reference_length': 1195018}

In [134]:
MODEL_NAME

'felipeoes/boto-7b'

In [135]:
# drop na and remove duplicates
df = df.dropna(subset=[MODEL_NAME])
df = df.drop_duplicates(subset=['question', 'answer'])
len(df)

12741

In [140]:
df[MODEL_NAME].isna().sum()

0

In [145]:
# drop na and remove duplicatesin df2
df2 = df2.dropna(subset=[MODEL_NAME_ORIG_LLAMA])
df2 = df2.drop_duplicates(subset=['question', 'answer'])
len(df2)

6510

In [146]:
# get the questions and answer intersects from both datasets
intersect_questions = set(df['question']).intersection(set(df2['question']))
len(intersect_questions)

3273

In [147]:
MODEL_NAME = "felipeoes/boto-7b"
MODEL_NAME_ORIG_LLAMA = "meta-llama/Llama-2-7b-hf"
# get answers from df1 based on intersect questions
df1_compare = pd.DataFrame(columns=['question', 'answer', MODEL_NAME])
df2_compare = pd.DataFrame(columns=['question', 'answer', MODEL_NAME_ORIG_LLAMA])

for question in tqdm(intersect_questions):
    df1_compare = pd.concat([df1_compare, df[df['question'] == question][['question', 'answer', MODEL_NAME]]])
    df2_compare = pd.concat([df2_compare, df2[df2['question'] == question][['question', 'answer', MODEL_NAME_ORIG_LLAMA]]])
    
# filter unique questions
df1_compare = df1_compare.drop_duplicates(subset=['question'])
df2_compare = df2_compare.drop_duplicates(subset=['question'])
df1_compare

  0%|          | 0/3273 [00:00<?, ?it/s]

100%|██████████| 3273/3273 [00:07<00:00, 440.21it/s]


,question,answer,felipeoes/boto-7b
6343,Quais são os membros da Segunda Comissão de Promoções?,"Os membros da Segunda Comissão de Promoções são:\n\n* **Um Almirante-de-Esquadra e quatro Vice-Almirantes, todos em comissão, e que integrem o Con...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais são os membros da Segunda Comissão de Promoções?\n\..."
1513,"O que o Capitão dos Portos ou Delegado das Capitanias pode fazer para impedir a saída, a entrada ou a permanência de uma embarcação em um porto?","O Capitão dos Portos ou Delegado das Capitanias pode impedir a saída, a entrada ou a permanência de uma embarcação em um porto, no interesse da Se...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nO que o Capitão dos Portos ou Delegado das Capitanias pod..."
5631,"Qual é o objetivo da Lei nº 2.698, de 27 de dezembro de 1955?","A Lei nº 2.698, de 27 de dezembro de 1955, tem o objetivo de dar aplicação à receita proveniente da diferença de preços entre os combustíveis e lu...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o objetivo da Lei nº 2.698, de 27 de dezembro de 1..."
5952,Qual é o objetivo da Taxa de Renovação da Marinha Mercante?,A Taxa de Renovação da Marinha Mercante foi criada com o objetivo de financiar a renovação da frota mercante brasileira. O produto da taxa é desti...,"Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o objetivo da Taxa de Renovação da Marinha Mercant..."
5808,Quais foram os recursos utilizados para financiar a construção do porto de São Francisco do Sul?,"Os recursos utilizados para financiar a construção do porto de São Francisco do Sul foram as taxas de 2% e de 0,7% ouro, arrecadadas pelas reparti...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram os recursos utilizados para financiar a const..."
...,...,...,...
1436,"Quais são as atividades industriais ou comerciais que estão sujeitas às obrigações da proporcionalidade previstas no Decreto-Lei nº 1.843, de 7 de...","As atividades industriais ou comerciais que estão sujeitas às obrigações da proporcionalidade previstas no Decreto-Lei nº 1.843, de 7 de dezembro ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais são as atividades industriais ou comerciais que est..."
5399,Qual é o prazo para que os ministérios executem o que está previsto no art. 7º da Lei nº 2.035/1908?,"A Lei nº 2.075/1953 autoriza o Poder Executivo a abrir um crédito especial de Cr$ 5.000.000,00 (cinco milhões de cruzeiros) para pagamento da part...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o prazo para que os ministérios executem o que est..."
5509,Quanto foi o valor orçado para a escola de aprendizes marinheiros do Estado de Minas Gerais em 1911?,"O valor orçado para a escola de aprendizes marinheiros do Estado de Minas Gerais em 1911 foi de 23.640$, sendo 22.940$ para o pessoal e 700$ para ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuanto foi o valor orçado para a escola de aprendizes mar..."
5906,Qual é o percentual do produto da taxa destinado ao Fundo de Marinha Mercante?,32%,"Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o percentual do produto da taxa destinado ao Fundo..."


In [149]:
df2_compare

,question,answer,meta-llama/Llama-2-7b-hf
6343,Quais são os membros da Segunda Comissão de Promoções?,"Os membros da Segunda Comissão de Promoções são:\n\n* **Um Almirante-de-Esquadra e quatro Vice-Almirantes, todos em comissão, e que integrem o Con...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais são os membros da Segunda Comissão de Promoções?\n\..."
1513,"O que o Capitão dos Portos ou Delegado das Capitanias pode fazer para impedir a saída, a entrada ou a permanência de uma embarcação em um porto?","O Capitão dos Portos ou Delegado das Capitanias pode impedir a saída, a entrada ou a permanência de uma embarcação em um porto, no interesse da Se...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nO que o Capitão dos Portos ou Delegado das Capitanias pod..."
5631,"Qual é o objetivo da Lei nº 2.698, de 27 de dezembro de 1955?","A Lei nº 2.698, de 27 de dezembro de 1955, tem o objetivo de dar aplicação à receita proveniente da diferença de preços entre os combustíveis e lu...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o objetivo da Lei nº 2.698, de 27 de dezembro de 1..."
2484,Qual é o objetivo da Taxa de Renovação da Marinha Mercante?,"O objetivo da Taxa de Renovação da Marinha Mercante é arrecadar recursos para o Fundo da Marinha Mercante, que é destinado ao desenvolvimento e à ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o objetivo da Taxa de Renovação da Marinha Mercant..."
5808,Quais foram os recursos utilizados para financiar a construção do porto de São Francisco do Sul?,"Os recursos utilizados para financiar a construção do porto de São Francisco do Sul foram as taxas de 2% e de 0,7% ouro, arrecadadas pelas reparti...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais foram os recursos utilizados para financiar a const..."
...,...,...,...
1436,"Quais são as atividades industriais ou comerciais que estão sujeitas às obrigações da proporcionalidade previstas no Decreto-Lei nº 1.843, de 7 de...","As atividades industriais ou comerciais que estão sujeitas às obrigações da proporcionalidade previstas no Decreto-Lei nº 1.843, de 7 de dezembro ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuais são as atividades industriais ou comerciais que est..."
5399,Qual é o prazo para que os ministérios executem o que está previsto no art. 7º da Lei nº 2.035/1908?,"A Lei nº 2.075/1953 autoriza o Poder Executivo a abrir um crédito especial de Cr$ 5.000.000,00 (cinco milhões de cruzeiros) para pagamento da part...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o prazo para que os ministérios executem o que est..."
5509,Quanto foi o valor orçado para a escola de aprendizes marinheiros do Estado de Minas Gerais em 1911?,"O valor orçado para a escola de aprendizes marinheiros do Estado de Minas Gerais em 1911 foi de 23.640$, sendo 22.940$ para o pessoal e 700$ para ...","Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQuanto foi o valor orçado para a escola de aprendizes mar..."
5906,Qual é o percentual do produto da taxa destinado ao Fundo de Marinha Mercante?,32%,"Responda à pergunta abaixo, forneça uma resposta completa e detalhada.\n\n### Pergunta:\nQual é o percentual do produto da taxa destinado ao Fundo..."


In [150]:
print(len(df1_compare))
print(len(df2_compare))

# check NA values
print(df1_compare[MODEL_NAME].isna().sum())
print(df2_compare[MODEL_NAME_ORIG_LLAMA].isna().sum())

3273
3273
0
0


In [160]:
# preprocess TARGET COL to remove repetition from text and split in END_TOKEN
START_RESPONSE = "### Resposta:"
END_TOKEN = "### Fim"

def preprocess_text(text):
    """ Get text in between START_RESPONSE and END_TOKEN """
    return text.split(START_RESPONSE)[1].split(END_TOKEN)[0].strip()

df1_compare[MODEL_NAME] = df1_compare[MODEL_NAME].apply(preprocess_text)
print(df1_compare[MODEL_NAME][0])

# preprocess TARGET COL to remove repetition from text and split in END_TOKEN
START_RESPONSE = "### Resposta:"
END_TOKEN_ORIG_LLAMA = "###"

def preprocess_text_orig_llama(text):
    """ Get text in between START_RESPONSE and END_TOKEN """
    return text.split(START_RESPONSE)[1].split(END_TOKEN_ORIG_LLAMA)[0].strip()

df2_compare[MODEL_NAME_ORIG_LLAMA] = df2_compare[MODEL_NAME_ORIG_LLAMA].apply(preprocess_text_orig_llama)
print(df2_compare[MODEL_NAME_ORIG_LLAMA][0])

As empresas que tiveram recursos improvídos pelo CONAMA na Decisão N ° 005, de 27 декабрь 2006 são:
 * TRANSMISSORA SUDESTE-NORDESTE
 * EIDAI DO BRASIL MADEIRAS S/A
 * COLUMBIA RIO CARNES LTDA
 * NOBLE DO BRASIL S/C LTDA
 * VALDIR APARECIDO BENTO
 * WARLI HAROLDO LUIZ CASTRO
 * LOURIVAL EVANGELISTA CERQUEIRA
 * PETROBRÁS PETRÓLEO BRASILEIRO S/A
 * PETROBRÁS TRANSPORTE S/A - TRANSPETRO
 * PETROBRÁS S/A
 * CARA CARAMBA PRODUTOS LTDA
 * QUEIROZ AGROINDUSTRIAL LTDA
 * SAMAD - SAPURI MA
O decreto-lei foi promulgado pelo governo brasileiro para estabelecer a criação da primeira instituição pública de ensino superior no Brasil, com o nome de Universidade Federal do Rio Grande do Sul (UFRGS). A UFRSG foi criada em 17 de julho de 1953, através da Lei Estadual n° 680/53.


In [161]:
# run rouge and bleu score on the intersect questions
REFERENCE_COL = "answer"
PREDICTION_COL = MODEL_NAME
PREDICTION_COL_ORIG_LLAMA = MODEL_NAME_ORIG_LLAMA

rouge_results = calculate_rouge(df1_compare[PREDICTION_COL], df1_compare[REFERENCE_COL])
bleu_results = calculate_bleu(df1_compare[PREDICTION_COL], df1_compare[REFERENCE_COL])

rouge_results

{'rouge1': 0.8135788416076144,
 'rouge2': 0.7542529365576192,
 'rougeL': 0.7994135619269539,
 'rougeLsum': 0.8055049083356886}

In [162]:
bleu_results

{'bleu': 0.6259797407957226,
 'precisions': [0.8383218842001963,
  0.7706617072457315,
  0.7353458317278521,
  0.7060811418094228],
 'brevity_penalty': 0.8225362941739115,
 'length_ratio': 0.8365661941300802,
 'translation_length': 232332,
 'reference_length': 277721}

In [163]:
rouge_results_orig_llama = calculate_rouge(df2_compare[PREDICTION_COL_ORIG_LLAMA], df2_compare[REFERENCE_COL])
bleu_results_orig_llama = calculate_bleu(df2_compare[PREDICTION_COL_ORIG_LLAMA], df2_compare[REFERENCE_COL])

rouge_results_orig_llama

{'rouge1': 0.2308832493244387,
 'rouge2': 0.061979877138664145,
 'rougeL': 0.15958478250715033,
 'rougeLsum': 0.18435780545489414}

In [164]:
bleu_results_orig_llama

{'bleu': 0.021221825245950485,
 'precisions': [0.20965394726928827,
  0.03124334617563284,
  0.010681038307641583,
  0.004334521099343429],
 'brevity_penalty': 0.9043341669428703,
 'length_ratio': 0.9086313620084598,
 'translation_length': 252187,
 'reference_length': 277546}

In [166]:
# create dataframe with results
df_results = pd.DataFrame(columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu'])
df_results = pd.concat([df_results, pd.DataFrame([['boto', rouge_results['rouge1'], rouge_results['rouge2'], rouge_results['rougeL'], bleu_results['bleu']], ['llama', rouge_results_orig_llama['rouge1'], rouge_results_orig_llama['rouge2'], rouge_results_orig_llama['rougeL'], bleu_results_orig_llama['bleu']]], columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu'])])
df_results

C:\Users\Felipe O E Santo\AppData\Local\Temp\ipykernel_16072\2386794531.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([['boto', rouge_results['rouge1'], rouge_results['rouge2'], rouge_results['rougeL'], bleu_results['bleu']], ['llama', rouge_results_orig_llama['rouge1'], rouge_results_orig_llama['rouge2'], rouge_results_orig_llama['rougeL'], bleu_results_orig_llama['bleu']]], columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu'])])


,model_name,rouge1,rouge2,rougeL,bleu
0,boto,0.813579,0.754253,0.799414,0.625980
1,llama,0.230883,0.061980,0.159585,0.021222


In [168]:
# create dataframe with results
df_results = pd.DataFrame(columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu', 'diff_boto_rouge1', 'diff_boto_rouge2', 'diff_boto_rougeL', 'diff_boto_bleu'])
df_results = pd.concat([df_results, pd.DataFrame([['boto', rouge_results['rouge1'], rouge_results['rouge2'], rouge_results['rougeL'], bleu_results['bleu'], 0, 0, 0, 0], ['llama', rouge_results_orig_llama['rouge1'], rouge_results_orig_llama['rouge2'], rouge_results_orig_llama['rougeL'], bleu_results_orig_llama['bleu'], rouge_results_orig_llama['rouge1'] - rouge_results['rouge1'], rouge_results_orig_llama['rouge2'] - rouge_results['rouge2'], rouge_results_orig_llama['rougeL'] - rouge_results['rougeL'], bleu_results_orig_llama['bleu'] - bleu_results['bleu']]], columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu', 'diff_boto_rouge1', 'diff_boto_rouge2', 'diff_boto_rougeL', 'diff_boto_bleu'])])
df_results

C:\Users\Felipe O E Santo\AppData\Local\Temp\ipykernel_16072\1775732088.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([['boto', rouge_results['rouge1'], rouge_results['rouge2'], rouge_results['rougeL'], bleu_results['bleu'], 0, 0, 0, 0], ['llama', rouge_results_orig_llama['rouge1'], rouge_results_orig_llama['rouge2'], rouge_results_orig_llama['rougeL'], bleu_results_orig_llama['bleu'], rouge_results_orig_llama['rouge1'] - rouge_results['rouge1'], rouge_results_orig_llama['rouge2'] - rouge_results['rouge2'], rouge_results_orig_llama['rougeL'] - rouge_results['rougeL'], bleu_results_orig_llama['bleu'] - bleu_results['bleu']]], columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu', '

,model_name,rouge1,rouge2,rougeL,bleu,diff_boto_rouge1,diff_boto_rouge2,diff_boto_rougeL,diff_boto_bleu
0,boto,0.813579,0.754253,0.799414,0.625980,0.000000,0.000000,0.000000,0.000000
1,llama,0.230883,0.061980,0.159585,0.021222,-0.582696,-0.692273,-0.639829,-0.604758


In [156]:
# # create dataframe with results
# df_results = pd.DataFrame(columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu', 'diff_boto_rouge1', 'diff_boto_rouge2', 'diff_boto_rougeL', 'diff_boto_bleu'])
# df_results = pd.concat([df_results, pd.DataFrame([['boto', rouge_results['rouge1'], rouge_results['rouge2'], rouge_results['rougeL'], bleu_results['bleu'], 0, 0, 0, 0], ['llama', rouge_results_orig_llama['rouge1'], rouge_results_orig_llama['rouge2'], rouge_results_orig_llama['rougeL'], bleu_results_orig_llama['bleu'], rouge_results_orig_llama['rouge1'] - rouge_results['rouge1'], rouge_results_orig_llama['rouge2'] - rouge_results['rouge2'], rouge_results_orig_llama['rougeL'] - rouge_results['rougeL'], bleu_results_orig_llama['bleu'] - bleu_results['bleu']]], columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu', 'diff_boto_rouge1', 'diff_boto_rouge2', 'diff_boto_rougeL', 'diff_boto_bleu'])])

C:\Users\Felipe O E Santo\AppData\Local\Temp\ipykernel_16072\2386794531.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([['boto', rouge_results['rouge1'], rouge_results['rouge2'], rouge_results['rougeL'], bleu_results['bleu']], ['llama', rouge_results_orig_llama['rouge1'], rouge_results_orig_llama['rouge2'], rouge_results_orig_llama['rougeL'], bleu_results_orig_llama['bleu']]], columns=['model_name', 'rouge1', 'rouge2', 'rougeL', 'bleu'])])


,model_name,rouge1,rouge2,rougeL,bleu
0,boto,0.521588,0.477334,0.497246,0.329314
1,llama,0.285633,0.131738,0.209098,0.072858


In [169]:
df_results.to_csv('results.csv', index=False)

# COMPARE QUALITATIVELY

In [170]:
# SELECT 50 RANDOM QUESTIONS FROM DF
random_df = df1_compare.sample(n=50, random_state=1)
random_df

,question,answer,felipeoes/boto-7b
188,Quais são os requisitos para a autorização de queima controlada de óleo no mar?,"Para obter autorização para queima controlada de óleo no mar, o respondedor deve atender aos seguintes requisitos:\n\n * Ter um Plano de Emergênci...","Para obter autorização para queima controlada de óleo nomar, o respondedor deve atender aos seguintes requisitos:\n\n * Ter um Plano de Emergência..."
1472,Qual é o órgão responsável pela distribuição e redistribuição dos créditos?,A Diretoria de Fazenda do Ministério da Marinha é o órgão responsável pela distribuição e redistribuição dos créditos.,A Diretoria de Fazenda do Ministério da Marinha é o órgão responsável pela distribUIção e redistribuição dos crÉditos.
1215,Quais são as classes de jazidas que podem ser autorizadas ou concedidas pelo CNP?,O CNP pode autorizar ou conceder jazidas das classes IX - rochas betuminosas e piro-betuminosas - e X - petróleo e gases naturais.,O CNP pode autorizar ou conceder jazidas das classes IX - rochas betuminosas e piro-betuminosas - e X - petróleo e gases naturais.
5672,Qual foi o valor total dos créditos abertos para o Ministério da Marinha?,"O valor total dos créditos abertos para o Ministério da Marinha foi de 1.704:402$517, sendo 1.000:000$000 em crédito extraordinário e 704:402$517 ...","O valor total dos créditos abertos для o Ministério da Marinha foi de 1.704:402$517, sendo 1.000:000$000 em crédito extraordinário e 704:402$587 e..."
5141,Qual é o aeroporto que receberá o maior investimento do crédito suplementar concedido à Infraero?,"O Aeroporto de São Paulo/Congonhas - Deputado Freitas Nobre, com um investimento de R$ 28.895.500","O Aeroporto de São Paulo/Congonhas - Deputado Freitas Nobre, com um investimento de R$ 28.895.500"
1114,Por que foi necessário estender as normas do Decreto-Lei nº 312/1938 às entidades autárquicas?,"O objetivo do Decreto-Lei nº 1.156/1939 é modificar a divisão administrativa e judiciária do Território do Acre, criando dois cargos de juiz de di...","O objetivo do Decreto-Lei nº 1.156/1939 é modificar a divisão administrativa e judiciária do Território do Acre, criando dois cargos de juiz de d..."
5550,"Qual é o valor do crédito especial autorizado pela Lei nº 2.381, de 27 de dezembro de 1954?","O valor do crédito especial autorizado pela Lei nº 2.381, de 27 de dezembro de 1954, é de Cr$ 360.000.000,00 (Trezentos e sessenta milhões de cruz...","O valor do crédito especial autorizado pelaLei nº 2.381 , de 27 de dezembro de 20th Century Fox Film Corporation, com sede em Hollywood, Califórni..."
768,Qual é a unidade de conservação federal que Karina de Oliveira Teixeira Sales foi designada para chefiar?,Karina de Oliveira Teixeira Sales foi designada para chefiar a Reserva Extrativista da Baía do Tubarão e Reserva Extrativista Itapetininga.,"Karina de Oliveira Teixeira Sales Foi Designada Para Chefar A Unidade De Conservação Federal Do Pará, Montanhas Da Serra Negra."
337,Quantos instrutores estão no quadro de instrutores do ICMBio?,"De acordo com a Portaria nº 192, de 09 de março de 2018, o quadro de instrutores do ICMBio é composto por 61 servidores, sendo 50 homens e 11 mulh...","De acordo com a Portaria nº 192, de 09 de março de 2018, o quadro de instrutores do IC MBIO é composto por 61 servidores, sendo 50 homens e 11 mul..."
414,Quais são as atribuições do Grupo de Assessoramento Técnico do PAN Aves Marinhas?,As atribuições do Grupo de Assessoramento Técnico do PAN Aves Marinhas são:\n\n * Acompanhar a implementação do PAN Aves Marinhas;\n * Realizar mo...,As atribuições do Grupo de Assessoriação Técnica do PAN Aves Marinhas são:\n * Acompanhar a implementação do PAN Aves Marinhas;\n * Realizar monit...


In [171]:
# add column with original llama answers
random_df[MODEL_NAME_ORIG_LLAMA] = df2_compare[MODEL_NAME_ORIG_LLAMA]
random_df

,question,answer,felipeoes/boto-7b,meta-llama/Llama-2-7b-hf
188,Quais são os requisitos para a autorização de queima controlada de óleo no mar?,"Para obter autorização para queima controlada de óleo no mar, o respondedor deve atender aos seguintes requisitos:\n\n * Ter um Plano de Emergênci...","Para obter autorização para queima controlada de óleo nomar, o respondedor deve atender aos seguintes requisitos:\n\n * Ter um Plano de Emergência...",Autorizar o uso da queima controlada de óleos na água é um processo complexo e exige compreender as leis federais e estaduais que regulam a poluiç...
1472,Qual é o órgão responsável pela distribuição e redistribuição dos créditos?,A Diretoria de Fazenda do Ministério da Marinha é o órgão responsável pela distribuição e redistribuição dos créditos.,A Diretoria de Fazenda do Ministério da Marinha é o órgão responsável pela distribUIção e redistribuição dos crÉditos.,"O crédito especial aberto no Banco Central tem um montante de R$ 10 bilhões, sendo que a taxa de juros aplicável é de 6% anual (com desconto). Ape..."
1215,Quais são as classes de jazidas que podem ser autorizadas ou concedidas pelo CNP?,O CNP pode autorizar ou conceder jazidas das classes IX - rochas betuminosas e piro-betuminosas - e X - petróleo e gases naturais.,O CNP pode autorizar ou conceder jazidas das classes IX - rochas betuminosas e piro-betuminosas - e X - petróleo e gases naturais.,"A Lei nº 13.486/2017 (Lei do Mineral) definiu os tipos de licenças mineradoras para o Brasil, sendo eles:\n- Licença de Prospecção;\n- Licença de ..."
5672,Qual foi o valor total dos créditos abertos para o Ministério da Marinha?,"O valor total dos créditos abertos para o Ministério da Marinha foi de 1.704:402$517, sendo 1.000:000$000 em crédito extraordinário e 704:402$517 ...","O valor total dos créditos abertos для o Ministério da Marinha foi de 1.704:402$517, sendo 1.000:000$000 em crédito extraordinário e 704:402$587 e...",O valor total de crédito abertos no período analisado é R$ 150 milhões. O valor atual do crédito é de R$ 267 milhões (R$ 439 milhão em 2018).
5141,Qual é o aeroporto que receberá o maior investimento do crédito suplementar concedido à Infraero?,"O Aeroporto de São Paulo/Congonhas - Deputado Freitas Nobre, com um investimento de R$ 28.895.500","O Aeroporto de São Paulo/Congonhas - Deputado Freitas Nobre, com um investimento de R$ 28.895.500",O Aeroporto Internacional de Manaus (MAO) será beneficiado com R$ 1 bilhão em obras de infraestrutura para a expansão da pista principal e constru...
1114,Por que foi necessário estender as normas do Decreto-Lei nº 312/1938 às entidades autárquicas?,"O objetivo do Decreto-Lei nº 1.156/1939 é modificar a divisão administrativa e judiciária do Território do Acre, criando dois cargos de juiz de di...","O objetivo do Decreto-Lei nº 1.156/1939 é modificar a divisão administrativa e judiciária do Território do Acre, criando dois cargos de juiz de d...","O Decreto-lei n° 312/1938 de 07 de Dezembro de 1938, estabeleceu o regime jurídico das organizações públicas no âmbito da República Portuguesa. A ..."
5550,"Qual é o valor do crédito especial autorizado pela Lei nº 2.381, de 27 de dezembro de 1954?","O valor do crédito especial autorizado pela Lei nº 2.381, de 27 de dezembro de 1954, é de Cr$ 360.000.000,00 (Trezentos e sessenta milhões de cruz...","O valor do crédito especial autorizado pelaLei nº 2.381 , de 27 de dezembro de 20th Century Fox Film Corporation, com sede em Hollywood, Califórni...","O crédito especial foi concedido por lei para a construção da Estrada de Ferro Transandina (EFT), que ligaria as cidades de Arica no Chile com La ..."
768,Qual é a unidade de conservação federal que Karina de Oliveira Teixeira Sales foi designada para chefiar?,Karina de Oliveira Teixeira Sales foi designada para chefiar a Reserva Extrativista da Baía do Tubarão e Reserva Extrativista Itapetininga.,"Karina de Oliveira Teixeira Sales Foi Designada Para Chefar A Unidade De Conservação Federa

In [173]:
# save csv
random_df.to_csv('compare_random_df.csv', index=False)